In [31]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
%matplotlib inline

np.set_printoptions(formatter={'all': lambda x: '%.4f' % x})

In [32]:
def gradient(fn: Callable, p: np.ndarray, delta) -> np.ndarray:
    fn_dx = lambda fn, p, delta: (fn(p[0] + delta, p[1]) - fn(p[0] - delta, p[1])) / (2 * delta)
    fn_dy = lambda fn, p, delta: (fn(p[0], p[1] + delta) - fn(p[0], p[1] - delta)) / (2 * delta)
    return np.array([fn_dx(fn, p, delta), fn_dy(fn, p, delta)])

def calc_alpha(fn, x, d, alpha):
    alphas = np.linspace(0, alpha, 20)
    min = np.argmin(np.array([fn(*(x + a * d)) for a in alphas]))
    return alphas[min]

def newton_dfp(
    fn: Callable,
    point: np.ndarray,
    delta: float = 10,
    alpha: float = 0.1,
    error: float = 0.001,
    max_iters: int = 1000,
    debug: bool = False
):
    x = point
    H = np.eye(len(x))
    epsilon = np.finfo(float).eps

    for iter in range(max_iters):
        grad = gradient(fn, x, delta)
        
        if grad == 0 or max_iters == 1:
            return { 'point': x, 'iters': iter + 1 }

        d_iter = np.matmul(-H, np.transpose(grad))
        alpha_iter = calc_alpha(fn, x, np.transpose(d_iter), alpha)

        x = x + alpha_iter * d_iter

        dx_iter = alpha_iter * d_iter
        delta_grad = gradient(fn, x, delta) - grad  # type: ignore
        
        H = H + (delta_grad * dx_iter) / (grad * dx_iter) - (np.matmul(H * grad, H * grad)) / (np.dot(np.dot(np.transpose(grad), H), delta_grad))
    
    return { 'point': x, 'iters': max_iters }

In [33]:
fn1 = lambda x, y : 100*(np.sqrt(x**2+(y+1)**2)-1)**2 + 90*(np.sqrt(x**2+(y+1)**2)-1)**2 -(20*x+40*y)
x = np.array([3, 3])

newton_dfp(
    fn1,
    x,
    alpha = 0.1,
    delta = 10,
    error = 0.001,
    max_iters = 1000,
    debug = True
)

/var/folders/5t/h61cy0q12yq1k0k5tf7ch0x40000gn/T/ipykernel_21287/579713451.py:38: RuntimeWarning: invalid value encountered in divide
  H = H + (delta_grad * dx_iter) / (grad * dx_iter) - (np.matmul(H * grad, H * grad)) / (np.dot(np.dot(np.transpose(grad), H), delta_grad))
/var/folders/5t/h61cy0q12yq1k0k5tf7ch0x40000gn/T/ipykernel_21287/579713451.py:38: RuntimeWarning: divide by zero encountered in divide
  H = H + (delta_grad * dx_iter) / (grad * dx_iter) - (np.matmul(H * grad, H * grad)) / (np.dot(np.dot(np.transpose(grad), H), delta_grad))


{'point': array([nan, nan]), 'iters': 1000}

In [34]:
# fn2 = lambda x, y : (x - 3) ** 2 + (y - 5) ** 2
# x = np.array([7, 9])

# levenberg_marquardt(
#     fn2,
#     x,
#     e1 = 0.001,
#     e2 = 0.002,
#     alpha = 0.001,
#     delta = 10
# )

In [35]:
# fn1 = lambda x, y: -25 * x + 110 * y
# x = np.array([7, 9])
# levenberg_marquardt(
#     fn1,
#     x,
#     e1 = 0.001,
#     e2 = 0.002,
#     alpha = 0.001,
#     delta = 10
# )

In [36]:
# fn3 = lambda x, y :190 * (np.sqrt(x ** 2 + (y + 1) ** 2) - 1) ** 2 - (20 * x + 40 * y)
# x = np.array([7, 9])

# levenberg_marquardt(
#     fn3,
#     x,
#     e1 = 0.001,
#     e2 = 0.002,
#     alpha = 0.001,
#     delta = 10
# )